## Develop a hand gesture recognition model that can accurately identify and classify different hand gestures from image or video data, enabling intuitive human-computer interaction and gesture-based control systems.

 # Data Preparation

In [2]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
IMG_SIZE = 128
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for subject_folder in os.listdir(folder_path):
        subject_path = os.path.join(folder_path, subject_folder)
        if os.path.isdir(subject_path):
            for gesture_folder in os.listdir(subject_path):
                gesture_path = os.path.join(subject_path, gesture_folder)
                if os.path.isdir(gesture_path):
                    for img_file in os.listdir(gesture_path):
                        img_path = os.path.join(gesture_path, img_file)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                        images.append(img)
                        labels.append(gesture_folder)  # Append gesture folder as the label
    return np.array(images), np.array(labels)
data_path = r"C:\Users\lalit\Downloads\archive (1)\leapGestRecog\leapGestRecog"
images, labels = load_images_from_folder(data_path)
print(f"Loaded {len(images)} images and {len(labels)} labels.")
if len(labels) == 0:
    raise ValueError("No labels found. Please check the folder structure and paths.")
print(f"Sample labels: {labels[:10]}")
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
print(f"Encoded labels: {labels_encoded[:10]}")

Loaded 20000 images and 20000 labels.
Sample labels: ['01_palm' '01_palm' '01_palm' '01_palm' '01_palm' '01_palm' '01_palm'
 '01_palm' '01_palm' '01_palm']
Encoded labels: [0 0 0 0 0 0 0 0 0 0]


#  Label Encoding

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

#  Train-Test Split

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

#  Model Design

In [7]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models

def build_cnn_model(input_shape):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Prevent overfitting
    model.add(layers.Dense(10, activation='softmax'))  # 10 gestures

    return model

cnn_model = build_cnn_model((IMG_SIZE, IMG_SIZE, 1))
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/385.0 MB 3.6 MB/s eta 0:01:46
   ---------------------------------------- 0.1/385.0 MB 3.6 MB/s eta 0:01:46
   ---------------------------------------- 0.1/385.0 MB 3.6 MB/s eta 0:01:46
   ---------------------------------------- 0.1/385.0 MB 3.6 MB/s eta 0:01:46
   ---------------------------------------- 0.3/385.0 MB 1.1 MB/s eta 0:05:43
   ---------------------------------------- 0.3/385.0 MB 1.3 MB/s eta 0:04:54
   ---------------------------------------- 0.5/385.0 MB 1.5 MB/s eta 0:04:15
   ---------------------------------------- 0.6/385.0 MB 1.7 MB/s eta 0:03:41
   ---------------------------------------- 0.8/385.0 MB 1.9 MB/s eta 0:03:21
   ---------------------------------------- 0.9/385.0 MB 2.0 MB/s eta 0:03:

C:\Users\lalit\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,354 (12.61 MB)

 Trainable params: 3,305,354 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

# Data Augmentation (Optional)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
X_train = X_train.reshape(-1, 128, 128, 1)  
datagen.fit(X_train)
datagen.fit(X_train) 

# Train the Model

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = cnn_model.fit(datagen.flow(X_train, y_train, batch_size=32),
                        epochs=5, validation_data=(X_test, y_test),
                        callbacks=[early_stopping])


Epoch 1/5


C:\Users\lalit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


500/500 ━━━━━━━━━━━━━━━━━━━━ 317s 621ms/step - accuracy: 0.1779 - loss: 3.4553 - val_accuracy: 0.7772 - val_loss: 0.6541
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 439s 876ms/step - accuracy: 0.6287 - loss: 1.0485 - val_accuracy: 0.8680 - val_loss: 0.3188
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 329s 654ms/step - accuracy: 0.7961 - loss: 0.6000 - val_accuracy: 0.9582 - val_loss: 0.1519
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 315s 626ms/step - accuracy: 0.8730 - loss: 0.3876 - val_accuracy: 0.9827 - val_loss: 0.0706
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 349s 696ms/step - accuracy: 0.9113 - loss: 0.2781 - val_accuracy: 0.9883 - val_loss: 0.0481


#  Evaluate the Model

In [16]:
test_loss, test_acc = cnn_model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')


125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 170ms/step - accuracy: 0.9867 - loss: 0.0517
Test accuracy: 0.99


#  Saving the Model

In [17]:
cnn_model.save('gesture_recognition_model.h5')


In [18]:
img_size = 224
gesture_classes = [
    '01_palm', '02_l', '03_fist', '04_fist_moved', 
    '05_thumb', '06_index', '07_ok', '08_palm_moved', 
    '09_c', '10_down'
]

# Load the Model

In [19]:
from tensorflow.keras.models import load_model

# Load the trained model
loaded_model = load_model('gesture_recognition_model.h5')


#  Test on a Single Image

In [21]:
import cv2
import numpy as np

# Function to preprocess an image (resize, normalize)
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image as grayscale
    img = cv2.resize(img, (128, 128))  # Resize image to match model input
    img = img / 255.0  # Normalize pixel values
    img = np.reshape(img, (1, 128, 128, 1))  # Reshape to (1, 128, 128, 1) for prediction
    return img

# Example image path (update this to the actual path of your test image)
test_img_path = r"C:\Users\lalit\Downloads\archive (1)\leapGestRecog\leapGestRecog\09\10_down\frame_09_10_0164.png"

# Preprocess the image
test_img = preprocess_image(test_img_path)

# Predict the gesture
prediction = loaded_model.predict(test_img)

# Decode the predicted class (assuming you used LabelEncoder during training)
predicted_class = np.argmax(prediction, axis=1)

print(f'Predicted class: {predicted_class}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 736ms/step
Predicted class: [9]
